# Synthetic Discovery Trace Generator

This notebook generates synthetic **Code Assistant** traces designed to stress test the 6 discovery question categories:

| Category | Description |
|----------|-------------|
| `themes` | General code quality, readability, best practices |
| `edge_cases` | Empty input, unicode, very long code, nested structures |
| `boundary_conditions` | Off-by-one errors, array bounds, null checks |
| `failure_modes` | Incorrect logic, missing error handling, security flaws |
| `missing_info` | Ambiguous requirements, missing context, unclear intent |
| `disagreements` | Multiple valid solutions, style preferences, trade-offs |

## Use Cases

1. **User Testing**: Generate traces for testing the assisted facilitation flow
2. **E2E Test Fixtures**: Export traces as JSON for automated tests
3. **DSPy Optimization**: Use coverage scores to self-improve generation

## 1. Setup

Install dependencies and configure credentials.

In [1]:
# Setup: Install dependencies if needed
# !pip install dspy mlflow pydantic rich

import json
import os
from datetime import datetime
from pathlib import Path
from typing import Optional

# Add parent directory to path for imports
import sys
sys.path.insert(0, str(Path.cwd().parent))

# Local imports
from notebooks.synthetic_trace_dspy import (
    SyntheticTrace,
    CategoryCoverageScore,
    get_generation_signatures,
    get_seed_traces,
    get_categories,
    build_lm,
    create_predictor,
    run_predictor,
)

# Display helpers
from rich import print as rprint
from rich.table import Table
from rich.console import Console

console = Console()

print("✅ Imports loaded successfully")
print(f"📋 Available categories: {get_categories()}")

✅ Imports loaded successfully
📋 Available categories: ['themes', 'edge_cases', 'boundary_conditions', 'failure_modes', 'missing_info', 'disagreements']


## 2. Configuration

Set up generation parameters and Databricks/MLflow credentials.

In [ ]:
# =============================================================================
# CONFIGURATION - Edit these values
# =============================================================================

# Generation settings
CONFIG = {
    # Number of traces to generate per category
    "traces_per_category": 2,
    
    # Difficulty distribution (should sum to 1.0)
    "difficulty_weights": {
        "easy": 0.3,
        "medium": 0.5,
        "hard": 0.2,
    },
    
    # Categories to generate (all by default)
    "target_categories": get_categories(),
    
    # Output settings
    "output_json_path": "../client/tests/fixtures/discovery-traces.json",
    "output_mlflow": True,  # Set to True to export to MLflow
    
    # Databricks/MLflow settings (only needed if output_mlflow=True)
    "databricks_host": os.getenv("DATABRICKS_HOST", "<your host here>"),
    "databricks_token": os.getenv("DATABRICKS_TOKEN", "<your token here>"),
    "model_endpoint": "databricks-gpt-5-2",  # Model serving endpoint
    "mlflow_experiment_name": "/Users/forrest.murray@databricks.com/synthetic-discovery-traces",
}

# Validate config
assert sum(CONFIG["difficulty_weights"].values()) == 1.0, "Difficulty weights must sum to 1.0"

rprint(f"[bold green]Configuration loaded:[/bold green]")
rprint(f"  • Traces per category: {CONFIG['traces_per_category']}")
rprint(f"  • Target categories: {CONFIG['target_categories']}")
rprint(f"  • Total traces to generate: {CONFIG['traces_per_category'] * len(CONFIG['target_categories'])}")
rprint(f"  • Output JSON: {CONFIG['output_json_path']}")
rprint(f"  • Output to MLflow: {CONFIG['output_mlflow']}")

Configuration loaded:

• Traces per category: 2

• Target categories: ['themes', 'edge_cases', 'boundary_conditions', 'failure_modes', 'missing_info', 
'disagreements']

• Total traces to generate: 12

• Output JSON: ../client/tests/fixtures/discovery-traces.json

• Output to MLflow: True

In [4]:
# Load and display seed examples
seed_traces = get_seed_traces()

# Create a summary table
table = Table(title="Seed Traces by Category")
table.add_column("Category", style="cyan")
table.add_column("Difficulty", style="magenta")
table.add_column("Input Preview", style="green", max_width=50)
table.add_column("Rationale", style="yellow", max_width=40)

for trace in seed_traces:
    input_preview = trace.input[:80].replace("\n", " ") + "..."
    table.add_row(
        ", ".join(trace.target_categories),
        trace.difficulty,
        input_preview,
        trace.rationale[:60] + "..." if len(trace.rationale) > 60 else trace.rationale,
    )

console.print(table)
print(f"\n📝 Total seed traces: {len(seed_traces)}")

                                              Seed Traces by Category                                              
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Category            ┃ Difficulty ┃ Input Preview                        ┃ Rationale                             ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ themes              │ easy       │ Review this function and suggest     │ Highlights general code quality       │
│                     │            │ improvements: ```python def          │ themes like naming, document...       │
│                     │            │ process(d):     r =...               │                                       │
│ edge_cases          │ medium     │ This JSON parser breaks on some      │ Shows edge cases like empty input and │
│                     │            │ inputs. Can you fix it? ```python    │ unicode BOM, while hin...             │
│                     │            │ import json  d...                    │                                       │
│ boundary_conditions │ easy       │ Is there a bug in this pagination    │ Classic off-by-one boundary condition │
│                     │            │ function? ```python def              │ with pagination indexi...             │
│                     │            │ get_page(items, page_n...            │                                       │
│ failure_modes       │ medium     │ Why does this SQL query sometimes    │ Security vulnerability (SQL           │
│                     │            │ return wrong results? ```python def  │ injection) is a critical failure...   │
│                     │            │ get_user(d...                        │                                       │
│ missing_info        │ medium     │ Write a function to validate an      │ The request lacks context about       │
│                     │            │ email address....                    │ validation depth, use case, ...       │
│ disagreements       │ hard       │ Should I use a class or functions    │ Classic design debate with no single  │
│                     │            │ for this data processing pipeline?   │ correct answer - depend...            │
│                     │            │ ```python #...                       │                                       │
└─────────────────────┴────────────┴──────────────────────────────────────┴───────────────────────────────────────┘


📝 Total seed traces: 6


In [15]:
import random

# =============================================================================
# Option A: Use seed traces only (no LLM required)
# =============================================================================

USE_LLM_GENERATION = True  # Set to True to use DSPy generation

if not USE_LLM_GENERATION:
    print("📦 Using pre-built seed traces (no LLM generation)")
    generated_traces = seed_traces.copy()
    
    # Optionally duplicate seeds to reach target count
    target_count = CONFIG["traces_per_category"] * len(CONFIG["target_categories"])
    while len(generated_traces) < target_count:
        # Add slightly modified copies
        for seed in seed_traces:
            if len(generated_traces) >= target_count:
                break
            # Create a variant by changing difficulty
            difficulties = ["easy", "medium", "hard"]
            new_difficulty = random.choice([d for d in difficulties if d != seed.difficulty])
            variant = SyntheticTrace(
                input=seed.input,
                output=seed.output,
                target_categories=seed.target_categories,
                difficulty=new_difficulty,
                rationale=f"[Variant] {seed.rationale}",
            )
            generated_traces.append(variant)
    
    print(f"✅ Total traces available: {len(generated_traces)}")

# =============================================================================
# Option B: Generate with DSPy + LLM
# =============================================================================

else:
    print("🤖 Generating traces with DSPy...")
    
    # Validate credentials
    if not CONFIG["databricks_host"] or not CONFIG["databricks_token"]:
        raise ValueError(
            "Databricks credentials required for LLM generation. "
            "Set DATABRICKS_HOST and DATABRICKS_TOKEN environment variables."
        )
    
    # Build LM and predictor
    lm = build_lm(
        endpoint_name=CONFIG["model_endpoint"],
        workspace_url=CONFIG["databricks_host"],
        token=CONFIG["databricks_token"],
        temperature=0.7,
    )
    
    signatures = get_generation_signatures()
    generator = create_predictor(signatures["GenerateSyntheticTrace"], lm)
    
    generated_traces = seed_traces.copy()
    previous_summaries = [t.rationale for t in seed_traces]
    
    # Generate additional traces
    for category in CONFIG["target_categories"]:
        for i in range(CONFIG["traces_per_category"]):
            # Sample difficulty based on weights
            difficulty = random.choices(
                list(CONFIG["difficulty_weights"].keys()),
                weights=list(CONFIG["difficulty_weights"].values()),
            )[0]
            
            print(f"  Generating: {category} ({difficulty})...")
            
            try:
                result = run_predictor(
                    generator,
                    lm,
                    target_categories=[category],
                    difficulty=difficulty,
                    previous_traces=previous_summaries[-10:],  # Last 10 for diversity
                )
                
                trace = result.trace
                generated_traces.append(trace)
                previous_summaries.append(trace.rationale)
                print(f"    ✅ Generated: {trace.rationale[:50]}...")
                
            except Exception as e:
                print(f"    ❌ Error: {e}")
    
    print(f"\n✅ Generated {len(generated_traces)} total traces")

🤖 Generating traces with DSPy...
  Generating: themes (medium)...
    ✅ Generated: Targets the “themes” category by focusing on maint...
  Generating: themes (medium)...
    ✅ Generated: Targets the “themes” category by focusing on maint...
  Generating: edge_cases (easy)...
    ✅ Generated: Focuses on edge cases in CSV parsing: escaped quot...
  Generating: edge_cases (easy)...
    ✅ Generated: Targets edge cases in string parsing: filenames wi...
  Generating: boundary_conditions (hard)...
    ✅ Generated: Targets boundary_conditions by focusing on inclusi...
  Generating: boundary_conditions (easy)...
    ✅ Generated: Targets boundary_conditions via an explicit inclus...
  Generating: failure_modes (medium)...
    ✅ Generated: Targets failure_modes by presenting a realistic we...
  Generating: failure_modes (easy)...
    ✅ Generated: This trace targets failure_modes by presenting a s...
  Generating: missing_info (medium)...
    ✅ Generated: The user request is intentionally undersp

## 5. Validation

Review the generated traces and their category coverage.

In [16]:
# Validate category coverage
from collections import Counter

category_counts = Counter()
difficulty_counts = Counter()

for trace in generated_traces:
    for cat in trace.target_categories:
        category_counts[cat] += 1
    difficulty_counts[trace.difficulty] += 1

# Display coverage summary
print("📊 Category Coverage:")
for cat in get_categories():
    count = category_counts.get(cat, 0)
    bar = "█" * count + "░" * (max(category_counts.values()) - count)
    print(f"  {cat:20} [{bar}] {count}")

print("\n📊 Difficulty Distribution:")
for diff in ["easy", "medium", "hard"]:
    count = difficulty_counts.get(diff, 0)
    pct = count / len(generated_traces) * 100
    print(f"  {diff:10} {count:3} ({pct:.0f}%)")

# Show a sample trace
print("\n" + "="*60)
print("📝 Sample Trace Preview:")
print("="*60)
sample = generated_traces[0]
print(f"Categories: {sample.target_categories}")
print(f"Difficulty: {sample.difficulty}")
print(f"Rationale: {sample.rationale}")
print(f"\nInput:\n{sample.input[:300]}...")
print(f"\nOutput:\n{sample.output[:300]}...")

📊 Category Coverage:
  themes               [███] 3
  edge_cases           [███] 3
  boundary_conditions  [███] 3
  failure_modes        [███] 3
  missing_info         [███] 3
  disagreements        [███] 3

📊 Difficulty Distribution:
  easy         6 (33%)
  medium      10 (56%)
  hard         2 (11%)

📝 Sample Trace Preview:
Categories: ['themes']
Difficulty: easy
Rationale: Highlights general code quality themes like naming, documentation, and readability

Input:
Review this function and suggest improvements:
```python
def process(d):
    r = []
    for i in d:
        if i > 0:
            r.append(i * 2)
    return r
```...

Output:
Here are some improvements for readability and clarity:

```python
def double_positive_numbers(numbers: list[int]) -> list[int]:
    """Return a list of positive numbers doubled."""
    return [num * 2 for num in numbers if num > 0]
```

Changes made:
- Descriptive function name (`double_positive_nu...


In [17]:
# =============================================================================
# Export to JSON (for E2E tests)
# =============================================================================

def export_to_json(traces: list[SyntheticTrace], output_path: str) -> str:
    """Export traces to JSON file in TraceUpload-compatible format."""
    output = Path(output_path)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    # Convert to TraceUpload-compatible format
    json_traces = []
    for trace in traces:
        json_traces.append({
            "input": trace.input,
            "output": trace.output,
            "context": {
                "target_categories": trace.target_categories,
                "difficulty": trace.difficulty,
                "rationale": trace.rationale,
                "source": "synthetic-discovery-generator",
                "generated_at": datetime.now().isoformat(),
            }
        })
    
    with open(output, "w") as f:
        json.dump(json_traces, f, indent=2)
    
    return str(output.resolve())

# Export JSON
json_path = export_to_json(generated_traces, CONFIG["output_json_path"])
print(f"✅ Exported {len(generated_traces)} traces to: {json_path}")

✅ Exported 18 traces to: /Users/forrest.murray/Documents/project-0xfffff/client/tests/fixtures/discovery-traces.json


In [ ]:
# =============================================================================
# Export to MLflow (optional - for workshop ingestion)
# =============================================================================

def export_to_mlflow(traces: list[SyntheticTrace], experiment_name: str) -> list[str]:
    """Export traces to MLflow as logged traces."""
    import mlflow
    
    # Set up MLflow
    mlflow.set_tracking_uri("databricks")
    mlflow.set_experiment(experiment_name)
    
    trace_ids = []
    
    for i, trace in enumerate(traces):
        # Create an MLflow trace for each synthetic trace
        with mlflow.start_span(name=f"synthetic_trace_{i}") as span:
            span.set_inputs({"user_input": trace.input})
            span.set_outputs({"assistant_output": trace.output})
            mlflow.update_current_trace({
                "target_categories": ",".join(trace.target_categories),
                "difficulty": trace.difficulty,
                "source": "synthetic-discovery-generator",
            })
            span.set_attributes({
                "rationale": trace.rationale,
            })
            trace_id = mlflow.get_active_trace_id()
            trace_ids.append(trace_id)
    
    return trace_ids

# Only export to MLflow if configured
if CONFIG["output_mlflow"]:
    if not CONFIG["databricks_host"] or not CONFIG["databricks_token"]:
        print("⚠️  Skipping MLflow export: Databricks credentials not configured")
    else:
        try:
            os.environ["DATABRICKS_HOST"] = CONFIG["databricks_host"]
            os.environ["DATABRICKS_TOKEN"] = CONFIG["databricks_token"]
            trace_ids = export_to_mlflow(generated_traces, CONFIG["mlflow_experiment_name"])
            print(f"✅ Exported {len(trace_ids)} traces to MLflow experiment: {CONFIG['mlflow_experiment_name']}")
        except Exception as e:
            print(f"❌ MLflow export failed: {e}")
else:
    print("ℹ️  MLflow export disabled. Set CONFIG['output_mlflow'] = True to enable.")

✅ Exported 18 traces to MLflow experiment: /Users/forrest.murray@databricks.com/synthetic-discovery-traces


2026/01/08 10:52:35 WARNING mlflow.tracing.export.mlflow_v3: Failed to send trace to MLflow backend: INVALID_PARAMETER_VALUE: Tag value for trace exceeds size threshold 255 bytes. Key is rationale, value is Targets the “themes” category by focusing on maintainability and code quality: clearer naming, modul...
2026/01/08 10:52:35 WARNING mlflow.tracing.export.mlflow_v3: Failed to send trace to MLflow backend: INVALID_PARAMETER_VALUE: Tag value for trace exceeds size threshold 255 bytes. Key is rationale, value is Targets edge cases in string parsing: filenames without dots, dotfiles, trailing dots, multiple dots...
2026/01/08 10:52:35 WARNING mlflow.tracing.export.mlflow_v3: Failed to send trace to MLflow backend: INVALID_PARAMETER_VALUE: Tag value for trace exceeds size threshold 255 bytes. Key is rationale, value is Targets the “themes” category by focusing on maintainability improvements: clearer naming, stronger ...
2026/01/08 10:52:35 WARNING mlflow.tracing.export.mlflow_v3: Failed

In [ ]:
# =============================================================================
# DSPy Optimization (Advanced - requires LLM)
# =============================================================================
# 
# This section demonstrates how to use DSPy optimizers to improve trace generation.
# Reference: https://dspy.ai/learn/optimization/overview/
#
# The ScoreTraceCoverage signature can serve as a metric function for:
# - BootstrapFewShot: Learn from examples that score well
# - MIPROv2: Optimize prompts using coverage as reward
#
# Example usage (requires Databricks credentials):
#
# ```python
# import dspy
# from notebooks.synthetic_trace_dspy import get_generation_signatures
#
# signatures = get_generation_signatures()
# GenerateSyntheticTrace = signatures["GenerateSyntheticTrace"]
# ScoreTraceCoverage = signatures["ScoreTraceCoverage"]
#
# # Define metric using coverage scorer
# def coverage_metric(example, prediction, trace=None):
#     scorer = dspy.Predict(ScoreTraceCoverage)
#     result = scorer(
#         trace=prediction.trace,
#         target_categories=example.target_categories,
#     )
#     return result.score.overall_score
#
# # Create training set from seed traces
# trainset = [
#     dspy.Example(
#         target_categories=t.target_categories,
#         difficulty=t.difficulty,
#         previous_traces=[],
#     ).with_inputs("target_categories", "difficulty", "previous_traces")
#     for t in seed_traces
# ]
#
# # Optimize the generator
# optimizer = dspy.BootstrapFewShot(metric=coverage_metric, max_bootstrapped_demos=4)
# optimized_generator = optimizer.compile(
#     dspy.Predict(GenerateSyntheticTrace),
#     trainset=trainset,
# )
#
# # Save optimized generator
# optimized_generator.save("optimized_trace_generator.json")
# ```

print("ℹ️  DSPy optimization example code is in the cell above.")
print("   Uncomment and run with Databricks credentials to optimize trace generation.")